In [9]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np 
import glob
from fbprophet import Prophet

In [11]:
PROPHET_PATH = '../data/prophet/'

In [12]:
glob.glob('../data/*')

['../data/prophet',
 '../data/sample_submission_1.csv',
 '../data/key_1.csv',
 '../data/train_1.csv']

In [13]:
full_df = pd.read_csv('../data/train_1.csv')

In [14]:
full_df.shape #days between dates is 550 so right

(145063, 551)

In [15]:
pageindexdf = full_df.loc[:,['Page']].reset_index()
pageindexdf = pageindexdf.rename(columns={'index':'page_index'}); pageindexdf.head()

,page_index,Page
0,0,2NE1_zh.wikipedia.org_all-access_spider
1,1,2PM_zh.wikipedia.org_all-access_spider
2,2,3C_zh.wikipedia.org_all-access_spider
3,3,4minute_zh.wikipedia.org_all-access_spider
4,4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...


In [16]:
pageindexdf.to_feather(PROPHET_PATH+'page_index.f')

In [17]:
pageindexdf.page_index.values

array([     0,      1,      2, ..., 145060, 145061, 145062])

In [18]:
pageindexdf.columns

Index(['page_index', 'Page'], dtype='object')

In [19]:
df = full_df.set_index('Page').T.reset_index(); df

Page,index,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,...,Drake_(músico)_es.wikipedia.org_all-access_spider,Skam_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Legión_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Doble_tentación_es.wikipedia.org_all-access_spider,Mi_adorable_maldición_es.wikipedia.org_all-access_spider,Underworld_(serie_de_películas)_es.wikipedia.org_all-access_spider,Resident_Evil:_Capítulo_Final_es.wikipedia.org_all-access_spider,Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider,Hasta_el_último_hombre_es.wikipedia.org_all-access_spider,Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider
0,2015-07-01,18.0,11.0,1.0,35.0,NaN,12.0,NaN,118.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-07-02,11.0,14.0,0.0,13.0,NaN,7.0,NaN,26.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-07-03,5.0,15.0,1.0,10.0,NaN,4.0,NaN,30.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-07-04,13.0,18.0,1.0,94.0,NaN,5.0,NaN,24.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-05,14.0,11.0,0.0,4.0,NaN,20.0,NaN,29.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-07-06,9.0,13.0,4.0,26.0,NaN,8.0,NaN,127.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-07-07,9.0,22.0,0.0,14.0,NaN,5.0,NaN,53.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-07-08,22.0,11.0,3.0,9.0,NaN,17.0,NaN,37.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-07-09,26.0,10.0,4.0,11.0,NaN,24.0,NaN,20.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-07-10,24.0,4.0,4.0,16.0,NaN,7.0,NaN,32.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Columns: 145064 entries, index to Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider
dtypes: float64(145063), object(1)
memory usage: 608.7+ MB


In [21]:
df.columns

Index(['index', '2NE1_zh.wikipedia.org_all-access_spider',
       '2PM_zh.wikipedia.org_all-access_spider',
       '3C_zh.wikipedia.org_all-access_spider',
       '4minute_zh.wikipedia.org_all-access_spider',
       '52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider',
       '5566_zh.wikipedia.org_all-access_spider',
       '91Days_zh.wikipedia.org_all-access_spider',
       'A'N'D_zh.wikipedia.org_all-access_spider',
       'AKB48_zh.wikipedia.org_all-access_spider',
       ...
       'Drake_(músico)_es.wikipedia.org_all-access_spider',
       'Skam_(serie_de_televisión)_es.wikipedia.org_all-access_spider',
       'Legión_(serie_de_televisión)_es.wikipedia.org_all-access_spider',
       'Doble_tentación_es.wikipedia.org_all-access_spider',
       'Mi_adorable_maldición_es.wikipedia.org_all-access_spider',
       'Underworld_(serie_de_películas)_es.wikipedia.org_all-access_spider',
       'Resident_Evil:_Capítulo_Final_es.wikipedia.org_all-access_spider',
       'Enamorándome_de_Ramó

In [22]:
df.columns = ['ds'] + [str(x) for x in pageindexdf.index.values.tolist()]

In [23]:
df.columns

Index(['ds', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '145053', '145054', '145055', '145056', '145057', '145058', '145059',
       '145060', '145061', '145062'],
      dtype='object', length=145064)

In [24]:
df['ds'] = pd.to_datetime(df.ds)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Columns: 145064 entries, ds to 145062
dtypes: datetime64[ns](1), float64(145063)
memory usage: 608.7 MB


In [26]:
df.head()

,ds,0,1,2,3,4,5,6,7,8,...,145053,145054,145055,145056,145057,145058,145059,145060,145061,145062
0,2015-07-01,18.0,11.0,1.0,35.0,NaN,12.0,NaN,118.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-07-02,11.0,14.0,0.0,13.0,NaN,7.0,NaN,26.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-07-03,5.0,15.0,1.0,10.0,NaN,4.0,NaN,30.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-07-04,13.0,18.0,1.0,94.0,NaN,5.0,NaN,24.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-05,14.0,11.0,0.0,4.0,NaN,20.0,NaN,29.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df

,ds,0,1,2,3,4,5,6,7,8,...,145053,145054,145055,145056,145057,145058,145059,145060,145061,145062
0,2015-07-01,18.0,11.0,1.0,35.0,NaN,12.0,NaN,118.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-07-02,11.0,14.0,0.0,13.0,NaN,7.0,NaN,26.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-07-03,5.0,15.0,1.0,10.0,NaN,4.0,NaN,30.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-07-04,13.0,18.0,1.0,94.0,NaN,5.0,NaN,24.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-07-05,14.0,11.0,0.0,4.0,NaN,20.0,NaN,29.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-07-06,9.0,13.0,4.0,26.0,NaN,8.0,NaN,127.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-07-07,9.0,22.0,0.0,14.0,NaN,5.0,NaN,53.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-07-08,22.0,11.0,3.0,9.0,NaN,17.0,NaN,37.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-07-09,26.0,10.0,4.0,11.0,NaN,24.0,NaN,20.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-07-10,24.0,4.0,4.0,16.0,NaN,7.0,NaN,32.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
dsdf = df.loc[:,['ds']]; print(dsdf.info()); dsdf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 1 columns):
ds    550 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.4 KB
None


,ds
0,2015-07-01
1,2015-07-02
2,2015-07-03
3,2015-07-04
4,2015-07-05


In [29]:
dsdf.to_feather(PROPHET_PATH+'ds.f')

In [30]:
pagedf = df.iloc[:,1:]; print(pagedf.info()); pagedf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Columns: 145063 entries, 0 to 145062
dtypes: float64(145063)
memory usage: 608.7 MB
None


,0,1,2,3,4,5,6,7,8,9,...,145053,145054,145055,145056,145057,145058,145059,145060,145061,145062
0,18.0,11.0,1.0,35.0,NaN,12.0,NaN,118.0,5.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.0,14.0,0.0,13.0,NaN,7.0,NaN,26.0,23.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,15.0,1.0,10.0,NaN,4.0,NaN,30.0,14.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13.0,18.0,1.0,94.0,NaN,5.0,NaN,24.0,12.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14.0,11.0,0.0,4.0,NaN,20.0,NaN,29.0,9.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pagedf.to_feather(PROPHET_PATH+'pagedf.f')